<a href="https://colab.research.google.com/github/GingerBoredMan/Character_Level_Text_Generation/blob/master/Character_Level_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [0]:
with open('anna.txt', 'r') as f:
  text = f.read()

In [5]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

In [0]:
chars = tuple(set(text))

In [0]:
int2char = dict(enumerate(chars))
char2int = {ch:ii for ii, ch in int2char.items()}

encoded = np.array([char2int[ch] for ch in text])

In [8]:
encoded [:100]

array([68, 38, 32,  8, 72, 12, 39, 82, 11, 54, 54, 54, 28, 32,  8,  8, 75,
       82, 69, 32, 58, 36,  1, 36, 12, 79, 82, 32, 39, 12, 82, 32,  1,  1,
       82, 32,  1, 36, 35, 12, 25, 82, 12, 76, 12, 39, 75, 82, 21, 77, 38,
       32,  8,  8, 75, 82, 69, 32, 58, 36,  1, 75, 82, 36, 79, 82, 21, 77,
       38, 32,  8,  8, 75, 82, 36, 77, 82, 36, 72, 79, 82, 31, 24, 77, 54,
       24, 32, 75, 60, 54, 54,  3, 76, 12, 39, 75, 72, 38, 36, 77])

In [0]:
def one_hot_encode(arr, n_labels):
    
  
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
   
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
   
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [0]:
def get_batches(arr, batch_size, seq_length):
 
    
    batch_size_total = batch_size * seq_length
    
    n_batches = len(arr)//batch_size_total
    
    arr = arr[:n_batches*batch_size_total]
    
    arr = arr.reshape((batch_size,-1))
    
    for n in range(0, arr.shape[1], seq_length):
       
        x = arr[:, n:n+seq_length]
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [11]:
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [0]:
class model(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
    
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
    
        self.lstm = nn.LSTM(input_size = len(self.chars), hidden_size = self.n_hidden, num_layers = self.n_layers,
                        dropout = self.drop_prob, batch_first = True)
    
        self.dropout = nn.Dropout(drop_prob)
    
        self.fc = nn.Linear(n_hidden, len(self.chars))
    
    def forward(self, x, hidden):
                
        r_output, hidden = self.lstm(x, hidden)
        
        out = self.dropout(r_output)
        out = out.contiguous().view(-1, self.n_hidden)
        
        out = self.fc(out)
        

        return out, hidden 
      
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
    

In [0]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            h = tuple([each.data for each in h])

            net.zero_grad()
            
            output, h = net(inputs, h)
            
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            if counter % print_every == 0:
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() 
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [14]:
n_hidden=512
n_layers=2

net = model(chars, n_hidden, n_layers)
print(net)

model(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


In [15]:
batch_size = 128
seq_length = 100
n_epochs = 20

train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/20... Step: 10... Loss: 3.2383... Val Loss: 3.1914
Epoch: 1/20... Step: 20... Loss: 3.1430... Val Loss: 3.1335
Epoch: 1/20... Step: 30... Loss: 3.1365... Val Loss: 3.1213
Epoch: 1/20... Step: 40... Loss: 3.1113... Val Loss: 3.1194
Epoch: 1/20... Step: 50... Loss: 3.1385... Val Loss: 3.1167
Epoch: 1/20... Step: 60... Loss: 3.1152... Val Loss: 3.1140
Epoch: 1/20... Step: 70... Loss: 3.1069... Val Loss: 3.1103
Epoch: 1/20... Step: 80... Loss: 3.1110... Val Loss: 3.0995
Epoch: 1/20... Step: 90... Loss: 3.0920... Val Loss: 3.0770
Epoch: 1/20... Step: 100... Loss: 3.0413... Val Loss: 3.0204
Epoch: 1/20... Step: 110... Loss: 2.9446... Val Loss: 2.9244
Epoch: 1/20... Step: 120... Loss: 2.8375... Val Loss: 2.8138
Epoch: 1/20... Step: 130... Loss: 2.7398... Val Loss: 2.7006
Epoch: 2/20... Step: 140... Loss: 2.6571... Val Loss: 2.6125
Epoch: 2/20... Step: 150... Loss: 2.5869... Val Loss: 2.5333
Epoch: 2/20... Step: 160... Loss: 2.5227... Val Loss: 2.4792
Epoch: 2/20... Step: 170... Loss:

In [0]:
model_name = 'rnn_20_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

In [0]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        h = tuple([each.data for each in h])
        out, h = net(inputs, h)

        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        return net.int2char[char], h

In [0]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)
    
    chars.append(char)
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)
    print(chars)
    return ''.join(chars)

In [33]:
print(sample(net, 1000, prime='Anna', top_k=5))

['A', 'n', 'n', 'a', ',', ' ', 't', 'e', 'l', 'l', 'i', 'n', 'g', ' ', 't', 'h', 'e', 'm', ' ', 't', 'h', 'e', ' ', 'm', 'o', 'n', 'e', 'y', '\n', 'a', 'n', 'd', ' ', 't', 'h', 'e', ' ', 'f', 'r', 'i', 'g', 'h', 't', 'e', 'n', 'e', 'd', ' ', 'w', 'o', 'r', 'd', 's', ',', ' ', 'a', 'n', 'd', ' ', 's', 't', 'r', 'u', 'c', 'k', ' ', 'h', 'i', 's', ' ', 'f', 'a', 'c', 'e', ',', ' ', 'h', 'e', ' ', 'h', 'a', 'd', ' ', 'b', 'e', 'g', 'i', 'n', 'n', 'e', 'd', ',', ' ', 'a', 'n', 'd', ' ', 't', 'h', 'a', 't', ' ', 'h', 'e', ' ', 'h', 'a', 'd', ' ', 'b', 'e', 'e', 'n', ' ', 't', 'h', 'e', ' ', 's', 'e', 'n', 's', 'e', ' ', 'o', 'f', ' ', 't', 'h', 'e', ' ', 'c', 'o', 'u', 'n', 't', 'r', 'y', "'", 's', ',', ' ', 't', 'h', 'e', ' ', 's', 't', 'r', 'i', 'n', 'g', ' ', 'w', 'o', 'u', 'l', 'd', ' ', 'b', 'e', ' ', 't', 'h', 'r', 'o', 'u', 'g', 'h', ' ', 'o', 'n', 'e', '\n', 's', 'i', 'n', 'c', 'e', ' ', 't', 'h', 'e', 'r', 'e', ' ', 'w', 'o', 'u', 'l', 'd', ' ', 'b', 'e', ' ', 't', 'o', ' ', 'h', 'a

In [36]:
with open('rnn_20_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = model(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [40]:
print(sample(loaded, 2000, top_k=5, prime="We're in the endgame now"))

['W', 'e', "'", 'r', 'e', ' ', 'i', 'n', ' ', 't', 'h', 'e', ' ', 'e', 'n', 'd', 'g', 'a', 'm', 'e', ' ', 'n', 'o', 'w', ',', '"', ' ', 's', 'h', 'e', ' ', 's', 'a', 'i', 'd', ',', ' ', 'w', 'a', 's', ' ', 't', 'a', 'l', 'k', 'i', 'n', 'g', ' ', 'a', 'b', 'o', 'u', 't', ' ', 'h', 'i', 'm', '.', '\n', '\n', '"', 'Y', 'o', 'u', '\n', 'c', 'a', 'n', 'n', 'o', 't', ' ', 'l', 'i', 'k', 'e', ' ', 'm', 'y', ' ', 'w', 'i', 'f', 'e', ' ', 'i', 's', ' ', 's', 'o', ' ', 't', 'a', 'k', 'e', 'n', ' ', 'u', 'p', ' ', 't', 'h', 'e', ' ', 'c', 'o', 'u', 'n', 't', 'r', 'y', ',', ' ', 'a', 'n', 'd', ' ', 't', 'h', 'i', 's', ' ', 's', 'a', 'y', 'i', 'n', 'g', ' ', 'i', 's', ' ', 'a', ' ', 'l', 'o', 'v', 'e', '.', '\n', '\n', 'A', ' ', 'm', 'a', 'n', ' ', 'o', 'f', ' ', 't', 'h', 'e', ' ', 'p', 'i', 's', 't', ' ', 'h', 'e', ' ', 'h', 'a', 'd', ' ', 'n', 'o', 't', ' ', 'b', 'e', 'e', 'n', ' ', 't', 'o', ' ', 's', 'p', 'a', 'r', 'k', ' ', 'o', 'f', ' ', 'm', 'y', ' ', 's', 'o', 'r', 'r', 'o', 'w', ',', ' ',